In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import MeanShift, estimate_bandwidth
import pymeanshift as pms
import os, sys
from glob import glob
from PIL import Image 
import keras
import tensorflow

## Resize images

In [ ]:
# imgs = glob('./hair_images/*')

# def resize(imgs):
#     for img in imgs:
#         filename = img.split('/')[2]
#         filename = filename.split('.')[0]
#         if filename == 'image_urls':
#             continue
#         if os.path.exists('./resized_hair_images/'+filename+'_resized.jpg') == True:
#             continue
#         im = Image.open(img)
#         imResize = im.resize((300,400), Image.ANTIALIAS)
#         imResize.save('./resized_hair_images/'+filename+'_resized.jpg', 'JPEG', quality=90)

# resize(imgs)

## Find hair via contours

In [ ]:
# min_YCrCb = np.array([0,133,77],np.uint8)
# max_YCrCb = np.array([255,173,127],np.uint8)

In [ ]:
# img = cv2.imread('wavy.jpg')

# imageYCrCb = cv2.cvtColor(img,cv2.COLOR_BGR2YCR_CB)
# skinRegion = cv2.inRange(imageYCrCb,min_YCrCb,max_YCrCb)

# contours, hierarchy = cv2.findContours(skinRegion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Draw the contour on the source image
# for i, c in enumerate(contours):
#     area = cv2.contourArea(c)
#     if area > 9000:
#         cv2.drawContours(img, contours, i, (0, 255, 0), 3)

# # Display the source image
# cv2.imwrite('output1.jpg', img)
# plt.figure(figsize=(12,12))
# plt.imshow(img[:,:,::-1])
# plt.show()

## Find faces

In [ ]:
# face_cascade = cv2.CascadeClassifier('/Users/joanna/anaconda2/envs/py37/lib/python3.7/site-packages/cv2/data/haarcascade_frontalface_default.xml')
# eye_cascade = cv2.CascadeClassifier('/Users/joanna/anaconda2/envs/py37/lib/python3.7/site-packages/cv2/data/haarcascade_eye.xml')

In [ ]:
# img = cv2.imread('hair.jpg')
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=3)

# for (x,y,w,h) in faces:
#     cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
#     roi_gray = gray[y:y+h, x:x+w]
#     roi_color = img[y:y+h, x:x+w]
#     eyes = eye_cascade.detectMultiScale(roi_gray,minSize=(100, 100), maxSize=(150,150))
#     for (ex,ey,ew,eh) in eyes:
#         cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

# cv2.imwrite('output2.jpg', img)
# plt.figure(figsize=(12,12))
# plt.imshow(img[:,:,::-1])
# plt.show()

In [ ]:
# img = cv2.imread('hair.jpg')
# img = cv2.cvtColor(img ,cv2.COLOR_BGR2GRAY)
# cv2.imwrite('blackwhite.jpg', img)

# (segmented_image, labels_image, number_regions) = pms.segment(img, spatial_radius=6, 
#                                                               range_radius=4.5, min_density=1500)


In [ ]:
# plt.figure(figsize=(12,12))
# plt.imshow(segmented_image)
# plt.show()

# cv2.imwrite('blackwhite_seg.jpg', segmented_image)

## Playing with edge detection

In [ ]:
# import cv2
# import cv2
# import numpy as np
# from matplotlib import pyplot as plt

In [ ]:
# img = cv2.imread('hair_images/903ff49749.jpg',0)
# edges = cv2.Canny(img,100,250)

# plt.figure(figsize=(20,20))
# plt.subplot(121),plt.imshow(img,cmap = 'gray')
# plt.title('Original Image'), plt.xticks([]), plt.yticks([])
# plt.subplot(122),plt.imshow(edges,cmap = 'gray')
# plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

# plt.show()


In [ ]:
# def auto_canny(image, sigma=0.33):
# 	# compute the median of the single channel pixel intensities
# 	v = np.median(image)
     
# 	# apply automatic Canny edge detection using the computed median
# 	lower = int(max(0, (1.0 - sigma) * v))
# 	upper = int(min(255, (1.0 + sigma) * v))
# 	edged = cv2.Canny(image, lower, upper)
 
# 	# return the edged image
# 	return edged

# imagePath = 'hair_images/903ff49749.jpg'
# image = cv2.imread(imagePath)
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# blurred = cv2.GaussianBlur(gray, (3, 3), 0)


In [ ]:

# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# blurred = cv2.GaussianBlur(gray, (3, 3), 0)
 
# # apply Canny edge detection using a wide threshold, tight
# # threshold, and automatically determined threshold
# wide = cv2.Canny(blurred, 10, 200)
# tight = cv2.Canny(blurred, 225, 250)
# auto = auto_canny(blurred)

In [ ]:
# plt.figure(figsize=(20,20))
# plt.subplot(121),plt.imshow(img,cmap = 'gray')
# plt.title('Original Image'), plt.xticks([]), plt.yticks([])
# plt.subplot(122),plt.imshow(wide,cmap = 'gray')
# plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

# plt.show()

## Classifying masked Reddit images via CNN

In [63]:
#import sys
#sys.path.append('./flaskexample/')

from model_image import predict_class

In [64]:
from glob import glob
import pandas as pd
import numpy as np
import os

In [ ]:
img_df = pd.read_csv('hair_images/image_urls.csv', sep='    ')

images = glob('masked_images/*')
img_df['class']=''

count = 0
for im in images:
    
    fil = './hair_images/'+im.split('/')[1]
    print(fil)
    
    row = img_df[img_df['file_path'].str.match(fil)]
    if row.empty:
        count += 1
        continue
    #print(os.getcwd())
    label = predict_class(im)
    
    ind = row.index[0]
    img_df['class'].loc[ind] = label

/Users/joanna/anaconda2/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


./hair_images/5b68119bf6.jpg
[1.9061925e-01 3.3091456e-01 9.4655304e+00 3.4889318e-03 2.0867041e-10
 3.4512698e-03 5.9947949e-03]
./hair_images/a2662b61ab.jpg
[1.8304473e+00 3.7954724e+00 3.2861528e+00 3.3009428e-01 2.2824588e-06
 4.7578338e-01 2.8204700e-01]
./hair_images/e21f702d40.jpg
[2.8748798e-01 1.3377348e-01 9.5767441e+00 9.1531190e-05 2.0989692e-12
 3.4840858e-05 1.8672692e-03]
./hair_images/70910f9b3f.jpg
[4.6976972e-03 3.4855530e-03 9.9917946e+00 5.4397216e-08 1.5450119e-13
 1.1952002e-06 2.2015185e-05]
./hair_images/f3ea01109a.jpg
[4.2408116e-02 5.6008432e-02 9.9003277e+00 4.4316536e-08 4.3289648e-14
 3.3391687e-05 1.2227884e-03]
./hair_images/7cbefea9ac.jpg
[1.2880259e-06 1.1694451e-04 9.9998817e+00 4.7954463e-10 2.4943724e-18
 4.3041034e-08 1.9488454e-08]
./hair_images/cf98b4f6e8.jpg
[4.4878017e-02 9.9922139e-03 9.9449081e+00 3.5741621e-06 7.8127295e-15
 4.9676426e-05 1.6887594e-04]
./hair_images/4976a63732.jpg
[1.1256295e-02 1.0903759e-03 9.9870920e+00 3.7435587e-08 4.69

[7.0716982e+00 1.8963134e+00 7.0752633e-01 7.7009290e-02 1.3776955e-05
 2.6143759e-02 2.2129607e-01]
./hair_images/586b187064.jpg
./hair_images/2ca83be897.jpg
[3.3960248e-07 1.6023208e-04 9.9998388e+00 5.4719822e-11 1.9219534e-22
 9.1023331e-08 5.1488132e-07]
./hair_images/90bf6f23fa.jpg
[1.9902607e-02 2.6275477e+00 2.9552299e-01 7.0356135e+00 2.4410568e-08
 1.3510026e-02 7.9031549e-03]
./hair_images/175b902267.jpg
[8.8825455e+00 4.7472209e-01 6.4155567e-01 1.0028898e-05 4.3642062e-16
 5.2745818e-06 1.1611601e-03]
./hair_images/a34298f565.jpg
./hair_images/c86ba2cd6b.jpg
[2.8807664e-04 3.6839795e-01 1.4441162e-02 9.3414679e+00 3.2766126e-03
 2.6779288e-01 4.3362980e-03]
./hair_images/b65757bfbc.jpg
[5.3326585e-05 9.9550123e+00 1.1295490e-03 4.3712687e-02 4.6127971e-12
 1.0188345e-07 9.1010617e-05]
./hair_images/8c37287cca.jpg
[8.2452246e-04 5.9178506e-05 9.9991169e+00 2.6895432e-07 7.8534902e-22
 1.1331311e-09 4.1186603e-08]
./hair_images/8eec686644.jpg
[2.8102482e-03 6.3635660e-03 4.4

[1.1540521e-01 4.0178686e-01 9.4691486e+00 6.6430708e-03 6.2785951e-08
 1.5821033e-04 6.8583288e-03]
./hair_images/5f97e42a0b.jpg
[2.5171562e-04 1.3992894e-01 9.8593721e+00 4.2249165e-10 3.4019387e-18
 4.3357713e-06 4.4296228e-04]
./hair_images/9bd736f5f5.jpg
./hair_images/bf1f8ad29d.jpg
[2.2673421e-02 9.5031395e+00 4.6152893e-01 3.6723915e-04 7.6347068e-11
 9.8084340e-05 1.2192147e-02]
./hair_images/cabbc82ac3.jpg
[1.3918493e-02 9.6577616e+00 2.7405005e-03 2.9913380e-04 2.3508188e-15
 1.7101379e-04 3.2510951e-01]
./hair_images/08b4de85cf.jpg
[8.3121210e-03 6.9610798e-01 9.2912016e+00 1.5463795e-03 1.3140351e-08
 3.7191410e-04 2.4604972e-03]
./hair_images/1730a12204.jpg
[3.6721134e-01 7.7876106e-02 9.5388727e+00 1.5899470e-02 2.2629627e-09
 1.2407532e-04 1.6429072e-05]
./hair_images/58b74c3a9a.jpg
[5.7601746e-02 8.7989464e+00 1.1406941e+00 4.0908300e-05 2.2920940e-17
 1.7577207e-06 2.7152721e-03]
./hair_images/6adf9c20ff.jpg
[3.7547057e-03 6.0087003e-02 9.9357815e+00 2.8206117e-04 4.84

[0.24474272 6.462074   0.10121995 2.5142405  0.01243094 0.11263297
 0.55265903]
./hair_images/33c49496b8.jpg
[6.6446757e-01 7.6533198e+00 1.9772431e-01 1.6852708e-01 1.2704396e-06
 2.6511615e-02 1.2894480e+00]
./hair_images/364cb6c7a5.jpg
[6.52415156e-06 1.75866589e-03 1.08610885e-02 6.59926367e+00
 5.82895475e-03 3.38226914e+00 1.18165635e-05]
./hair_images/9b8e6b47c3.jpg
[5.1715603e+00 3.7451866e-01 4.1916699e+00 4.8689105e-02 7.8320973e-06
 1.5319999e-01 6.0354158e-02]
./hair_images/faf91f1b43.jpg
[7.9755759e-05 1.1978549e-01 9.8749371e+00 1.4794347e-07 1.6397770e-13
 4.9692644e-03 2.2795613e-04]
./hair_images/622c3d1348.jpg
[3.9350312e-02 9.3045912e+00 3.4376938e-02 6.1934310e-01 3.4530933e-07
 5.4678283e-05 2.2836884e-03]
./hair_images/20c02f97c7.jpg
./hair_images/bd47e8e926.jpg
./hair_images/e53043e792.jpg
[8.5893720e-03 3.9417511e-03 9.9873838e+00 8.5923501e-05 3.9776563e-12
 2.2147029e-07 4.2767162e-08]
./hair_images/e06bb9aa73.jpg
[3.6199953e-02 2.1147215e+00 9.6178867e-02 7.6

[7.8633185e-03 4.3740125e+00 1.8128464e-03 5.2099266e+00 6.9306296e-08
 2.8093359e-01 1.2545107e-01]
./hair_images/9e5f17076d.jpg
[6.4484948e-01 5.6419425e+00 3.6657972e+00 2.4847095e-03 4.4725570e-09
 2.2225548e-03 4.2704135e-02]
./hair_images/e4c870860d.jpg
[5.4257627e+00 1.5072758e+00 2.2189715e+00 8.3586723e-01 3.1759421e-06
 3.4689261e-03 8.6504919e-03]
./hair_images/c03f4ba580.jpg
[1.7977046e-04 1.3940850e-01 2.1090826e-02 9.8391228e+00 7.0218880e-09
 3.6661157e-07 1.9799510e-04]
./hair_images/612cf132cb.jpg
[6.6088347  1.1337821  0.1892181  1.3104736  0.00841125 0.1145726
 0.63470787]
./hair_images/bf92dc0768.jpg
[1.3204658e-03 2.2904584e+00 6.8189526e+00 8.8353312e-01 2.7816200e-06
 3.6777486e-03 2.0551684e-03]
./hair_images/f2c4838a9a.jpg
[2.0745428e-01 1.0684658e+00 8.6952391e+00 6.1355240e-07 1.0308334e-15
 1.8608196e-04 2.8651932e-02]
./hair_images/365791634f.jpg
./hair_images/927e89cd20.jpg
[4.6269153e-04 5.6522368e-03 9.9937954e+00 5.1498595e-05 1.1465136e-10
 1.8518942e-

[4.1035004e-04 3.9080961e-04 9.9986038e+00 5.9369166e-04 5.1377613e-10
 6.6193985e-07 5.0998483e-07]
./hair_images/d67b8f875a.jpg
[9.7975826e+00 5.6741592e-03 1.9617812e-01 4.0367836e-06 4.9187593e-15
 8.0876460e-05 4.8136312e-04]
./hair_images/b99133b264.jpg
[0.33625877 1.0603459  7.17091    0.05845644 0.05793994 1.0655524
 0.25053698]
./hair_images/f1735db86f.jpg
./hair_images/739ac1d2d9.jpg
[1.50934    1.20781    0.75039595 1.3271556  2.3614013  1.6061337
 1.2377639 ]
./hair_images/5789e03e0b.jpg
[1.7644507e-01 6.2535703e-01 8.6541027e-02 9.0429726e+00 7.9028180e-04
 3.9727651e-03 6.3921444e-02]
./hair_images/8d4197cee2.jpg
[5.2381004e-04 6.6452837e-01 9.3052282e+00 4.3011714e-05 3.8003101e-10
 2.3951204e-02 5.7233563e-03]
./hair_images/bb679cafae.jpg
[1.30502880e+00 3.18930268e-01 8.08615494e+00 1.09833636e-07
 1.81465943e-16 8.51688150e-04 2.89034307e-01]
./hair_images/a1c3ad71a4.jpg
[7.6426835e+00 1.3477328e+00 9.4749463e-01 2.2089353e-02 1.0663425e-10
 2.5316086e-04 3.9746467e-0

[1.8123550e-02 1.9875406e-01 1.0146321e-01 9.6609917e+00 1.7127577e-02
 2.6384047e-03 9.0183195e-04]
./hair_images/fdfb07cc47.jpg
[9.0006894e-01 1.6977210e+00 4.9272709e+00 2.3767605e+00 4.8029460e-06
 7.1565300e-02 2.6608987e-02]
./hair_images/f62ad41cfd.jpg
./hair_images/d22252be02.jpg
[4.3643980e+00 2.7251835e+00 1.9902079e+00 6.5718973e-01 9.2151466e-05
 1.1773141e-01 1.4519711e-01]
./hair_images/893d189055.jpg
[1.3284607e-01 8.6315873e-04 9.8662825e+00 7.3621891e-06 3.9431911e-14
 7.6514618e-07 4.1215176e-07]
./hair_images/c28474cd9b.jpg
[2.1128353e-02 8.3327264e-01 2.0687916e+00 6.3436675e+00 3.4135953e-03
 7.0799643e-01 2.1729652e-02]
./hair_images/7a15303c65.jpg
[6.4334559e+00 5.8973515e-01 2.1206257e+00 2.6035228e-01 6.9182519e-05
 3.0762732e-02 5.6499845e-01]
./hair_images/9ce0dfcddb.jpg
./hair_images/4145f31ca3.jpg
[5.3908177e-02 7.9149812e-01 5.8781981e+00 4.5591900e-05 5.7111483e-05
 2.7774322e+00 4.9886000e-01]
./hair_images/415c7d9bde.jpg
[1.6964495e-01 1.7631873e-01 9.6

In [54]:
img_df.head(24)

,image_url,file_path,class
0,http://imgur.com/a/Qo6NW,not downloaded,
1,https://www.reddit.com/r/curlyhair/comments/4l...,not downloaded,
2,http://i.imgur.com/tZJAaPv.jpg,./hair_images/ce88676d03.jpg,curly
3,http://imgur.com/gd5lj1b,not downloaded,
4,https://imgur.com/a/Ttc96,not downloaded,
5,http://imgur.com/a/Qo6NW,not downloaded,
6,https://www.reddit.com/r/curlyhair/comments/4l...,not downloaded,
7,http://i.imgur.com/tZJAaPv.jpg,./hair_images/ce88676d03.jpg,
8,http://imgur.com/gd5lj1b,not downloaded,
9,https://imgur.com/a/Ttc96,not downloaded,


In [ ]:
print(count)